In [2]:
import pandas as pd
import numpy as np
import pickle

import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize, word_tokenize 

from sklearn.feature_extraction.text import TfidfVectorizer 

Idea: Match claims in reason dataset with IBM CoPA's by topic, then compare cosine similarities using avg W2V embeddings. Map topic to CoPA subject to embeddings, and one df to keep track of each topic in Reason.

In [3]:
pickle_copadict = open("./principle_argument_CoPA/PA_dict.pkl","rb")
principle_args = pickle.load(pickle_copadict)
print(principle_args)

{'Adolescent Rights': ['Many adolescents cannot make responsible decisions', 'Adolescents are as capable as adults'], 'Animal Rights': ['Animals should not be treated as property', 'There is nothing wrong with using animals to further human interests'], 'Big government': ['Public utility is best served by actions coordinated by central government', 'Public interest is best served and propelled by voluntary interactions,and not ones dictated by government'], 'Black market': ['Prohibiting products and activities makes them less visible and available, and thus less harmful', 'Prohibition is counterproductive and only leads to increased demand'], 'Clean energy': ['Humanity must embrace clean energy in order to fight climate change', 'Ecological concerns add further strain on the economy'], 'Coercion': ['A decisive and enforced policy is the best way to deliver a message', 'Enforcement tends to be less effective than persuasion and education'], 'Conservatism': ['The current system is workin

In [4]:
copa_df = pd.read_csv("./principle_argument_CoPA/IBM_Debater_(R)_CoPA-Motion-ACL-2019.v0.csv")
copa_df

,Action,Topic,Wikipedia title,Motion phrasing,Initial list?,# CoPAs,Adolescent rights,Animal rights,Big government,Black market,...,Right to privacy,Self-determination,Sexual morality,Subsidies,Technology,Tradition,Value of science,Virtual life,Wealth distribution,Welfare state
0,mandatory,Abstinence-only sex education,Abstinence-only sex education,Abstinence-only sex education should be mandatory,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,end,affirmative action,Affirmative action,We should end affirmative action,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,increase,airport racial profiling in the United States,Airport racial profiling in the United States,We should increase airport racial profiling in...,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,ban,alcoholic beverages,Alcoholic drink,We should ban alcoholic beverages,1.0,4.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,abolish,the American Jobs Act,American Jobs Act,We should abolish the American Jobs Act,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import pathlib
pathlib.Path().absolute()

PosixPath('/Users/joshy/cs577/577project-gitrepo/data')

In [6]:
wv_from_bin = KeyedVectors.load_word2vec_format("./word2vec/GoogleNews-vectors-negative300.bin",limit=50000, binary=True)

In [10]:

# complete_df = pd.read_pickle('reasons_complete_df.pkl')

In [11]:
# complete_df.head()

,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity,claim_SBERT_embedding
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974...","[0.021533493, -0.16520593, 0.35402358, 0.43329..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005...","[0.47159326, 0.4925952, -0.23349553, -0.087148..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553...","[0.4615807, 0.68479395, 0.737707, 0.26567188, ..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575...","[0.78423434, 0.56576806, 0.69618547, 0.0808582..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834...","[-0.19426306, 0.5660061, -0.13704692, -0.07844..."


In [8]:
try:
    reasons_df = pd.read_pickle('reasons_df.pkl')
except (OSError, IOError) as e:
    reasons_df = pd.read_csv("./reason-dataset.csv")
reasons_df.head()


,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834..."


In [9]:
docs = reasons_df['line'].tolist()
docs[:2]

['And if it is not possible for your to have a baby, due to shortage of money or time... Nowadays many couples would adopt unwanted babies.',
 'I believe that abortion cannot be justified because it is killing an innocent human being.']

In [8]:
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True, sublinear_tf=True)
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)

In [9]:
(tfidf_vectorizer_vectors.shape[0])

2848

In [10]:
# # this is just for show, not necessary
# # get the first vector out (for the first document)
# first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
# # place tf-idf values in a pandas data frame
# df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
# df.sort_values(by=["tfidf"],ascending=False)

In [11]:
tfidf_vocab = tfidf_vectorizer.vocabulary_
type(tfidf_vocab)

dict

In [12]:
tfidf_vectorizer_vectors.shape

(2848, 5810)

In [13]:
'may' in tfidf_vocab.keys()

True

In [14]:
def get_avg_vector_tfidf(w2v_model, vocab, words):
    # remove out-of-vocabulary words
    words = [vocab[word] * w2v_model[word] for word in words if (word in w2v_model.vocab and word in vocab.keys())]
    if len(words) >= 1:
        return np.mean(words, axis=0)
    else:
        return []

In [15]:
def embed_sentence(s, vocab):
    if s[-1] == '.':
        s=s[:-1]
    words = s.split()
    return get_avg_vector_tfidf(wv_from_bin, vocab, words)
test = (embed_sentence('It is a fact that there are differences between people. Hence, there should sometimes be differences in the way people are treated.', tfidf_vocab))


In [16]:
(np.mean(test, axis=0)).shape

()

In [17]:
CoPAs = pd.read_pickle(r'principle_argument_CoPA/PA_list.pkl')
CoPAs[:2]

['Many adolescents cannot make responsible decisions',
 'Adolescents are as capable as adults']

In [18]:
# settings that you use for count vectorizer will go here
tfidf_vectorizer_copa=TfidfVectorizer(use_idf=True, sublinear_tf=True)
# just send in all your docs here
tfidf_vectorizer_copa_vectors=tfidf_vectorizer_copa.fit_transform(CoPAs)
tfidf_vectorizer_copa_vectors.shape
tfidf_copa_vocab = tfidf_vectorizer_copa.vocabulary_


In [19]:
tfidf_copa_vocab

{'many': 210,
 'adolescents': 11,
 'cannot': 48,
 'make': 207,
 'responsible': 304,
 'decisions': 81,
 'are': 20,
 'as': 21,
 'capable': 49,
 'adults': 12,
 'animals': 18,
 'should': 324,
 'not': 233,
 'be': 31,
 'treated': 368,
 'property': 282,
 'there': 356,
 'is': 186,
 'nothing': 234,
 'wrong': 402,
 'with': 395,
 'using': 376,
 'to': 364,
 'further': 145,
 'human': 162,
 'interests': 182,
 'public': 290,
 'utility': 377,
 'best': 38,
 'served': 320,
 'by': 46,
 'actions': 6,
 'coordinated': 71,
 'central': 51,
 'government': 150,
 'interest': 181,
 'and': 17,
 'propelled': 281,
 'voluntary': 384,
 'interactions': 180,
 'ones': 243,
 'dictated': 90,
 'prohibiting': 278,
 'products': 276,
 'activities': 7,
 'makes': 208,
 'them': 354,
 'less': 196,
 'visible': 383,
 'available': 25,
 'thus': 361,
 'harmful': 154,
 'prohibition': 279,
 'counterproductive': 73,
 'only': 244,
 'leads': 194,
 'increased': 174,
 'demand': 85,
 'humanity': 163,
 'must': 224,
 'embrace': 110,
 'clean': 56

In [40]:
CoPAs_embedding = [] # works
for arg in CoPAs:
    CoPAs_embedding.append(embed_sentence(arg.lower(), tfidf_copa_vocab))

copa_embeds_tfidf_df = pd.DataFrame({'CoPAs': CoPAs, 'CoPA_embedding': CoPAs_embedding})
copa_embeds_tfidf_df.to_pickle('./principle_argument_CoPA/PA_embeds_tfidf_df.pkl')

In [23]:
def cosine(u, v):
        return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
def measure_pa_similarity(dataframe):
#     CoPAs_embedding = [] # works
#     for arg in CoPAs:
#         CoPAs_embedding.append(embed_sentence(arg.lower(), tfidf_copa_vocab))
# #     print(CoPAs_embedding)
    claim_embeddings = []
    similarity_scores = []
    for index, row in dataframe.iterrows():
        sentence=row['line']
        claim_embedding = embed_sentence(sentence.lower(), tfidf_vocab)
#         print('claim_embedding ',claim_embedding)
        claim_embeddings.append(claim_embedding)
        similarity = []
        for arg in CoPAs_embedding:
            similarity.append(cosine(claim_embedding, arg))
        similarity_scores.append(similarity)
    # similarity_scores is a list of a list of 74 embeddings for each line
    # len(similarity_scores[0]) ==  74
    dataframe['claim_tf_idf_embedding'] = pd.Series(claim_embeddings)
    dataframe['tf_idf_pa_similarity'] = pd.Series(similarity_scores)
    
    return dataframe, CoPAs_embedding
reasons_df, copa_embeddings = measure_pa_similarity(reasons_df)

In [24]:
reasons_df.head()

,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834..."


In [25]:
reasons_df['tf_idf_pa_similarity']

0       [0.48416767, 0.4241976, 0.54075974, 0.69915974...
1       [0.47884166, 0.43939564, 0.5063248, 0.70720005...
2       [0.43071118, 0.45206025, 0.48450536, 0.5497553...
3       [0.44407952, 0.4239362, 0.49705744, 0.70402575...
4       [0.41161808, 0.39982748, 0.43759474, 0.6643834...
                              ...                        
2843    [0.5100023, 0.41690615, 0.553481, 0.7322623, 0...
2844    [0.39890993, 0.26361057, 0.3582708, 0.5272109,...
2845    [0.39314574, 0.4079126, 0.45908293, 0.6267522,...
2846    [0.48969442, 0.34099227, 0.43148765, 0.6171639...
2847    [0.44804606, 0.35858345, 0.51951706, 0.5623813...
Name: tf_idf_pa_similarity, Length: 2848, dtype: object

In [26]:
reasons_df.to_pickle('reasons_df.pkl')

In [12]:
reasons_df = pd.read_pickle('reasons_df.pkl')
reasons_df.head()

,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity,avg_w2v_embedding
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974...","[0.5661276, 0.5389883, 0.6098048, 0.7101158, 0..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005...","[0.44384283, 0.46450335, 0.62090343, 0.7182656..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553...","[0.41737673, 0.5605264, 0.58117366, 0.53856754..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575...","[0.40784317, 0.45864588, 0.5202222, 0.669624, ..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834...","[0.46884528, 0.42420495, 0.49323082, 0.6081152..."


In [28]:
tfidf_copa_vocab

{'many': 210,
 'adolescents': 11,
 'cannot': 48,
 'make': 207,
 'responsible': 304,
 'decisions': 81,
 'are': 20,
 'as': 21,
 'capable': 49,
 'adults': 12,
 'animals': 18,
 'should': 324,
 'not': 233,
 'be': 31,
 'treated': 368,
 'property': 282,
 'there': 356,
 'is': 186,
 'nothing': 234,
 'wrong': 402,
 'with': 395,
 'using': 376,
 'to': 364,
 'further': 145,
 'human': 162,
 'interests': 182,
 'public': 290,
 'utility': 377,
 'best': 38,
 'served': 320,
 'by': 46,
 'actions': 6,
 'coordinated': 71,
 'central': 51,
 'government': 150,
 'interest': 181,
 'and': 17,
 'propelled': 281,
 'voluntary': 384,
 'interactions': 180,
 'ones': 243,
 'dictated': 90,
 'prohibiting': 278,
 'products': 276,
 'activities': 7,
 'makes': 208,
 'them': 354,
 'less': 196,
 'visible': 383,
 'available': 25,
 'thus': 361,
 'harmful': 154,
 'prohibition': 279,
 'counterproductive': 73,
 'only': 244,
 'leads': 194,
 'increased': 174,
 'demand': 85,
 'humanity': 163,
 'must': 224,
 'embrace': 110,
 'clean': 56

USELESS USELESS USELESS IGNORE IGNORE IGNORE BELOW THIS POINT

In [29]:
docs_wds = [word for line in docs for word in line.split()]

In [30]:
docs_2nd = [line.split() for line in docs]
docs_2nd

[['And',
  'if',
  'it',
  'is',
  'not',
  'possible',
  'for',
  'your',
  'to',
  'have',
  'a',
  'baby,',
  'due',
  'to',
  'shortage',
  'of',
  'money',
  'or',
  'time...',
  'Nowadays',
  'many',
  'couples',
  'would',
  'adopt',
  'unwanted',
  'babies.'],
 ['I',
  'believe',
  'that',
  'abortion',
  'cannot',
  'be',
  'justified',
  'because',
  'it',
  'is',
  'killing',
  'an',
  'innocent',
  'human',
  'being.'],
 ['Moreover,',
  'United',
  'Nations',
  'declaration',
  'says',
  'children',
  'have',
  'a',
  'right',
  'to',
  'protected',
  'before',
  'as',
  'well',
  'as',
  'after',
  'birth!'],
 ['Even',
  'if',
  'it',
  "doesn't",
  'have',
  'a',
  'brain,',
  'my',
  'belief',
  'is',
  'that',
  'the',
  'PERSON',
  'still',
  'has',
  'a',
  'soul.',
  'The',
  'soul',
  'is',
  'what',
  'gives',
  'the',
  'PERSON',
  'LIFE'],
 ["Can't",
  'you',
  'use',
  'a',
  'condom',
  'while',
  'having',
  'sex.',
  'Yes,',
  'having',
  'the',
  'baby',
  '

In [31]:
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix

tf = TfidfVectorizer(analyzer='word', ngram_range=(1,6),
                     min_df = 0, stop_words = 'english', sublinear_tf=True)
tfidf_matrix =  tf.fit_transform(docs)
feature_names = tf.get_feature_names()



In [32]:
feature_names[:3]

['00', '00 lives', '00 lives year']

In [33]:
doc = 0
feature_index = tfidf_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])

In [34]:
tfidf_matrix[doc,:].nonzero()[1]

array([90537, 60167, 81878,  7733, 69945, 62451, 90536, 60166, 81877,
        7732, 69944, 19854, 62450, 90535, 60165, 81876,  7731, 69943,
        3716, 19853, 62449, 90534, 60164, 81875,  7730, 69942, 93208,
        3715, 19844, 62448, 90533, 60160, 81874,  7729, 69941,  7438,
       93207,  3618, 19843, 62447, 90381, 59922, 81868,  7518, 69940],
      dtype=int32)

In [35]:
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
  print(w, s)

time nowadays couples adopt unwanted babies 0.15823430926109197
money time nowadays couples adopt unwanted 0.15823430926109197
shortage money time nowadays couples adopt 0.15823430926109197
baby shortage money time nowadays couples 0.15823430926109197
possible baby shortage money time nowadays 0.15823430926109197
nowadays couples adopt unwanted babies 0.15823430926109197
time nowadays couples adopt unwanted 0.15823430926109197
money time nowadays couples adopt 0.15823430926109197
shortage money time nowadays couples 0.15823430926109197
baby shortage money time nowadays 0.15823430926109197
possible baby shortage money time 0.15823430926109197
couples adopt unwanted babies 0.15823430926109197
nowadays couples adopt unwanted 0.15823430926109197
time nowadays couples adopt 0.15823430926109197
money time nowadays couples 0.15823430926109197
shortage money time nowadays 0.15823430926109197
baby shortage money time 0.15823430926109197
possible baby shortage money 0.15823430926109197
adopt unw